docker run -p 9000:8080 -p 6006:6006 asia-docker.pkg.dev/colab-images/public/runtime

In [2]:
# Initialize
!git clone --recursive https://github.com/mitosagi/puzzdra-nnsolver
%cd /kaggle/working/puzzdra-nnsolver
# %cd /content/puzzdra-nnsolver
!pip install --log=pip_log -e .
!pip install torchsummary

Cloning into 'puzzdra-nnsolver'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 305 (delta 34), reused 0 (delta 0), pack-reused 245
Receiving objects: 100% (305/305), 8.32 MiB | 27.57 MiB/s, done.
Resolving deltas: 100% (176/176), done.
Submodule 'extern/pybind11' (https://github.com/pybind/pybind11) registered for path 'extern/pybind11'
Cloning into '/kaggle/working/puzzdra-nnsolver/extern/pybind11'...
remote: Enumerating objects: 27240, done.        
remote: Counting objects: 100% (12/12), done.        
remote: Compressing objects: 100% (10/10), done.        
remote: Total 27240 (delta 2), reused 4 (delta 1), pack-reused 27228        
Receiving objects: 100% (27240/27240), 10.31 MiB | 17.66 MiB/s, done.
Resolving deltas: 100% (19261/19261), done.
Submodule path 'extern/pybind11': checked out '8de7772cc72daca8e947b79b83fea46214931604'
/kaggle/working/puzzdra-nnsolver
Obtaining file:

In [35]:
import random
import numpy as np
from puzzpy import PuzzTable

drop_color = 3
board_width = 6
board_height = 5

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'

def npUint8(array):
    return np.array(array, dtype=np.uint8)

def np_float(array):
    return np.array(array, dtype=np.float32)

class PuzzBoard():
    def reset(self, finish = False):
        self.prev_action = 255

        while True:
            self.table = PuzzTable("".join([str(random.randrange(drop_color)) for i in range(board_width*board_height)]), random.randrange(board_width), random.randrange(board_height), 50) # n色陣　操作時間m秒
            if self.table.eval_otoshi() == 0:
                break
                
        if finish:
                matrix = npUint8([[7 , 19, 10, 22, 13, 25],
                         [ 8, 20, 11, 23, 14, 26],
                         [ 9, 21, 12, 24, 15, 27],
                         [28, 29, 30, 4, 5, 6],
                         [1, 2, 3, 16, 17, 18]]).ravel() - 1
                
                arr = np.sort(npUint8([random.randrange(drop_color) for i in range(board_width*board_height)])).reshape([10, 3]).ravel()[matrix]                
                self.table = PuzzTable("".join([str(n) for n in arr]), random.randrange(board_width), random.randrange(board_height), 50) # n色陣　操作時間m秒
        
        return npUint8(self.table.get_table())

    def step(self, peek = False, force_action = None):
        next_tables = self.table.next_tables()
        valid_actions = [action for action, table in enumerate(next_tables) if table.get_table()[0][0] != 127 and abs(action - self.prev_action) != 2]
                    
        if not peek:
            if force_action != None:
                if force_action in valid_actions:
                    self.prev_action = force_action
                else:
                    self.prev_action = random.choice(valid_actions)
            else:
                self.prev_action = random.choice(valid_actions)
            self.table = next_tables[self.prev_action]

        return self.prev_action, npUint8([(npUint8(table.get_table()) if action in valid_actions else np.zeros_like(npUint8(table.get_table()))) for action, table in enumerate(next_tables)])
    def render(self):
        tcolor = [bcolors.RED, bcolors.BLUE, bcolors.GREEN, bcolors.MAGENTA, bcolors.YELLOW, bcolors.BLACK]
        start = self.table.get_XY_as_table()
        table = self.table.get_table()
        for i in range(board_height):
            for j in range(board_width):
                if start[i][j] == 1:
                    print(tcolor[table[i][j]-1]  +  bcolors.UNDERLINE + "●" + bcolors.ENDC, end='')
                else:
                    print(tcolor[table[i][j]-1]  + "●" + bcolors.ENDC, end='')
            print('')

In [36]:
env = PuzzBoard()
obs = env.reset(True)
env.render()

n_steps = 3

for step in range(n_steps):
    print("Step {}".format(step + 1))
    action, obs = env.step()
    with np.printoptions(threshold=np.inf):
        print("action: ", action)
#         print(obs)
        env.render()

●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
Step 1
action:  2
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
Step 2
action:  2
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
Step 3
action:  2
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●


In [4]:
def make_data(num_step, env):
    start = env.reset()

    sample_step = random.randrange(1, num_step + 1)

    for i in range(sample_step):
        answer, obs = env.step()
    sample_obs = obs
    sample_answer = answer
    
    for i in range(num_step - sample_step):
        answer, obs = env.step()
    end = obs[answer]

    return np.concatenate([npUint8([start]),sample_obs,npUint8([end])]), sample_answer
def make_data_fast(length):
    env = PuzzBoard()
    return [make_data(50, env) for i in range(length)]

# from multiprocessing import Pool
# process = 10
# p = Pool(process)
# %time result = p.map(make_data_fast, [1_000_000 // process  for i in range(process)]) # 100万データ生成に5分22秒かかる

# import itertools
# sample_data = npUint8([sample[0] for sample in itertools.chain(*result)])
# sample_labels = npUint8([sample[1] for sample in itertools.chain(*result)])
# np.save('sample_data_50', sample_data)
# np.save('sample_labels_50', sample_labels)

In [5]:
sample_data = np.load('/kaggle/input/puzz-data-50/sample_data_50.npy')
sample_labels = np.load('/kaggle/input/puzz-data-50/sample_labels_50.npy')
from torch.utils.data import Dataset
class PuzzDataSet(Dataset):
    def __init__(self, x, y):
        self.data = np.array(x, dtype=np.float32)
        self.labels = np.array(np.identity(4)[y], dtype=np.float32) # 4 for actions

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

dataset = PuzzDataSet(sample_data[0:len(sample_data) - 1_000],sample_labels[0:len(sample_labels) - 1_000])
print("dataset: ", dataset.__len__())
print("dataset: ", dataset.__getitem__(6))
testset = PuzzDataSet(sample_data[len(sample_data) - 1_000:],sample_labels[len(sample_labels) - 1_000:])
print("testset: ", testset.__len__())

dataset:  999000
dataset:  (array([[[2., 3., 2., 3., 3., 1.],
        [3., 2., 3., 1., 1., 3.],
        [2., 3., 3., 1., 3., 3.],
        [3., 1., 2., 3., 3., 2.],
        [1., 1., 2., 2., 1., 2.]],

       [[3., 2., 3., 3., 1., 1.],
        [2., 2., 3., 1., 3., 2.],
        [3., 3., 3., 3., 3., 1.],
        [2., 1., 1., 3., 1., 2.],
        [3., 1., 3., 2., 2., 2.]],

       [[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]],

       [[3., 2., 3., 3., 1., 1.],
        [2., 2., 3., 1., 3., 2.],
        [3., 3., 3., 3., 3., 1.],
        [2., 3., 1., 1., 1., 2.],
        [3., 1., 3., 2., 2., 2.]],

       [[3., 2., 3., 3., 1., 1.],
        [2., 2., 3., 1., 3., 2.],
        [3., 3., 3., 3., 3., 1.],
        [2., 1., 3., 1., 1., 2.],
        [3., 1., 3., 2., 2., 2.]],

       [[2., 3., 3., 3., 1., 1.],
        [3., 2., 1., 3., 3., 2.],
        [2., 3., 3., 3., 3., 1.],
        [2

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torchvision.models import resnet18, resnet34, resnet50, resnext50_32x4d

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_input_channel = 6
        num_classes = 4
        resnet = resnet34()
        resnet.conv1 = nn.Conv2d(num_input_channel, 64, kernel_size=7, stride=2, padding=3,bias=False)
        resnet.fc = nn.Linear(512, num_classes) # resnet18 or 34
#         resnet.fc = nn.Linear(2048, num_classes) # resnet50
        self.resnet = resnet
    def forward(self, x):
        x = F.interpolate(x, size=None, scale_factor=2, mode='nearest')
        x = self.resnet(x)
        return x

model = Net()
# for i, param in enumerate(model.parameters()):
#     param.requires_grad = True if i >= 0 else False
# #     param.requires_grad = True if i >= 45 else False
#     print(len(param), param.requires_grad)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("device: ", device)
model = model.to(device)
# print(summary(model, (6, 5, 6)))

train_loader = torch.utils.data.DataLoader(dataset, batch_size=200, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=200, shuffle=True)

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01)
# optimizer = optim.SGD(model.parameters(), lr=0.01,
#                       momentum=0.9, weight_decay=0.0001)
optimizer = optim.Adam(model.parameters())

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

import datetime

def train(epoch):
    total_loss = 0
    total_size = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            now = datetime.datetime.now()
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                now,
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), total_loss / total_size))
            writer.add_scalar('Loss/train', total_loss / total_size, epoch)
def test(epoch):
    total_loss = 0
    total_size = 0
    model.eval()
    for batch_idx, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        if batch_idx % 100 == 0:
            now = datetime.datetime.now()
            print('[{}] Test Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                now,
                epoch, batch_idx * len(data), len(test_loader.dataset),
                100. * batch_idx / len(test_loader), total_loss / total_size))
            writer.add_scalar('Loss/test', total_loss / total_size, epoch)
            break

for epoch in range(1, 10 + 1):
    train(epoch)
    test(epoch)

device:  cuda
[2023-11-04 07:22:06.995003] Train Epoch: 1 [0/999000 (0%)]	Average loss: 0.008436
[2023-11-04 07:22:24.629632] Train Epoch: 1 [20000/999000 (2%)]	Average loss: 0.004815
[2023-11-04 07:22:42.276486] Train Epoch: 1 [40000/999000 (4%)]	Average loss: 0.004574
[2023-11-04 07:22:59.929637] Train Epoch: 1 [60000/999000 (6%)]	Average loss: 0.004475
[2023-11-04 07:23:17.587239] Train Epoch: 1 [80000/999000 (8%)]	Average loss: 0.004434
[2023-11-04 07:23:35.252135] Train Epoch: 1 [100000/999000 (10%)]	Average loss: 0.004403
[2023-11-04 07:23:52.915320] Train Epoch: 1 [120000/999000 (12%)]	Average loss: 0.004377
[2023-11-04 07:24:10.553729] Train Epoch: 1 [140000/999000 (14%)]	Average loss: 0.004356
[2023-11-04 07:24:28.207295] Train Epoch: 1 [160000/999000 (16%)]	Average loss: 0.004344
[2023-11-04 07:24:45.892123] Train Epoch: 1 [180000/999000 (18%)]	Average loss: 0.004333
[2023-11-04 07:25:03.577065] Train Epoch: 1 [200000/999000 (20%)]	Average loss: 0.004328
[2023-11-04 07:25:21.

KeyboardInterrupt: 

In [18]:
torch.save(model, 'model_puzz_ressnet.pth')

In [ ]:
model = torch.load('/kaggle/input/puzz-resnet-weight/model_puzz_ressnet.pth', map_location=torch.device('cpu'))

In [25]:
from scipy.special import softmax
model.eval()

infer_board = PuzzBoard()
tmp_board = infer_board.reset().ravel()
start = tmp_board.reshape([5,6])
end = np.sort(tmp_board).reshape([5,6])
print(start)
print(end)
infer_board.render()
for i in range(100):
     
    data = np_float([np_float(np.concatenate([npUint8([start]),infer_board.step(peek=True)[1],npUint8([end])]))])
    action = model(torch.from_numpy(data).to(device)).tolist()[0]
    np.set_printoptions(suppress=True, precision=2, floatmode="fixed")
#     print(softmax(action))
    print(action.index(max(action)))
    infer_board.step(force_action = action.index(max(action)))
    
print("step: ", i)
infer_board.render()

[[2 1 2 3 2 2]
 [3 3 1 1 2 1]
 [2 3 3 2 1 2]
 [3 1 3 3 2 3]
 [3 2 1 1 2 2]]
[[1 1 1 1 1 1]
 [1 1 2 2 2 2]
 [2 2 2 2 2 2]
 [2 2 3 3 3 3]
 [3 3 3 3 3 3]]
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
0
3
0
3
0
3
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
step:  99
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
